In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-w1xmq5sa
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-w1xmq5sa
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-hz7q0v9o/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
%%cuda
#include <cuda_runtime.h>
#include <iostream>
#include <fstream>

// CUDA kernel for converting CSR to MTX
__global__ void csrToMtxKernel(const int *rowPtr, const int *colIdx, const float *values,
                               int nnz, int rows, int *outputRow, int *outputCol, float *outputVal) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < nnz) {
        int row = 0;
        while (rowPtr[row + 1] <= idx) {
            row++;
        }
        outputRow[idx] = row + 1; // 1-based index for MTX
        outputCol[idx] = colIdx[idx] + 1; // 1-based index for MTX
        outputVal[idx] = values[idx];
    }
}

void csrToMtx(const int *rowPtr, const int *colIdx, const float *values, int nnz, int rows) {
    int *d_rowPtr, *d_colIdx, *d_outputRow, *d_outputCol;
    float *d_values, *d_outputVal;

    cudaMalloc(&d_rowPtr, (rows + 1) * sizeof(int));
    cudaMalloc(&d_colIdx, nnz * sizeof(int));
    cudaMalloc(&d_values, nnz * sizeof(float));
    cudaMalloc(&d_outputRow, nnz * sizeof(int));
    cudaMalloc(&d_outputCol, nnz * sizeof(int));
    cudaMalloc(&d_outputVal, nnz * sizeof(float));

    cudaMemcpy(d_rowPtr, rowPtr, (rows + 1) * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_colIdx, colIdx, nnz * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_values, values, nnz * sizeof(float), cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocks = (nnz + threadsPerBlock - 1) / threadsPerBlock;
    csrToMtxKernel<<<blocks, threadsPerBlock>>>(d_rowPtr, d_colIdx, d_values, nnz, rows, d_outputRow, d_outputCol, d_outputVal);

    int *outputRow = new int[nnz];
    int *outputCol = new int[nnz];
    float *outputVal = new float[nnz];
    cudaMemcpy(outputRow, d_outputRow, nnz * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(outputCol, d_outputCol, nnz * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(outputVal, d_outputVal, nnz * sizeof(float), cudaMemcpyDeviceToHost);

    std::ofstream outFile("output.mtx");
    outFile << "%%MatrixMarket matrix coordinate real general\n";
    outFile << rows << " " << rows << " " << nnz << "\n";
    for (int i = 0; i < nnz; ++i) {
        outFile << outputRow[i] << " " << outputCol[i] << " " << outputVal[i] << "\n";
    }
    outFile.close();

    cudaFree(d_rowPtr);
    cudaFree(d_colIdx);
    cudaFree(d_values);
    cudaFree(d_outputRow);
    cudaFree(d_outputCol);
    cudaFree(d_outputVal);
    delete[] outputRow;
    delete[] outputCol;
    delete[] outputVal;
}

int main() {
    int rowPtr[] = {0, 1, 2, 3, 5};
    int colIdx[] = {2, 0, 1, 0, 3};
    float values[] = {3, 22, 17, 8, 10};
    int nnz = 5;
    int rows = 4;

    csrToMtx(rowPtr, colIdx, values, nnz, rows);
    return 0;
}


In [ ]:
%%cuda
#include <iostream>
#include <fstream>
#include <vector>
#include <sstream>
#include <cuda_runtime.h>

// Kernel to construct the CSR row pointer array
__global__ void mtxToCsrKernel(const int *rowIndices, int nnz, int *rowPtr, int rows) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < nnz) {
        atomicAdd(&rowPtr[rowIndices[idx] + 1], 1);
    }

    __syncthreads();

    // Prefix sum to build the rowPtr array
    if (idx == 0) {
        for (int i = 1; i <= rows; ++i) {
            rowPtr[i] += rowPtr[i - 1];
        }
    }
}

// Function to parse the MTX file
void parseMtxFile(const char *filename, std::vector<int> &rowIndices, std::vector<int> &colIndices, std::vector<float> &values, int &rows, int &cols, int &nnz) {
    std::ifstream file(filename);
    std::string line;

    // Skip headers
    while (std::getline(file, line)) {
        if (line[0] != '%') break;
    }

    std::istringstream ss(line);
    ss >> rows >> cols >> nnz;

    rowIndices.resize(nnz);
    colIndices.resize(nnz);
    values.resize(nnz);

    for (int i = 0; i < nnz; ++i) {
        int row, col;
        float val;
        file >> row >> col >> val;
        rowIndices[i] = row - 1; // Adjust for 0-based indexing
        colIndices[i] = col - 1;
        values[i] = val;
    }
}

void mtxToCsr(const char *filename) {
    int rows, cols, nnz;
    std::vector<int> rowIndices, colIndices;
    std::vector<float> values;

    parseMtxFile(filename, rowIndices, colIndices, values, rows, cols, nnz);

    int *d_rowIndices, *d_colIndices, *d_rowPtr;
    float *d_values;
    cudaMalloc(&d_rowIndices, nnz * sizeof(int));
    cudaMalloc(&d_colIndices, nnz * sizeof(int));
    cudaMalloc(&d_values, nnz * sizeof(float));
    cudaMalloc(&d_rowPtr, (rows + 1) * sizeof(int));

    cudaMemcpy(d_rowIndices, rowIndices.data(), nnz * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_colIndices, colIndices.data(), nnz * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_values, values.data(), nnz * sizeof(float), cudaMemcpyHostToDevice);

    cudaMemset(d_rowPtr, 0, (rows + 1) * sizeof(int));

    int threadsPerBlock = 256;
    int blocks = (nnz + threadsPerBlock - 1) / threadsPerBlock;
    mtxToCsrKernel<<<blocks, threadsPerBlock>>>(d_rowIndices, nnz, d_rowPtr, rows);

    int *rowPtr = new int[rows + 1];
    int *colIndicesOut = new int[nnz];
    float *valuesOut = new float[nnz];

    cudaMemcpy(rowPtr, d_rowPtr, (rows + 1) * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(colIndicesOut, d_colIndices, nnz * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(valuesOut, d_values, nnz * sizeof(float), cudaMemcpyDeviceToHost);

    // Output the CSR format to verify
    std::cout << "rowPtr: ";
    for (int i = 0; i <= rows; ++i) std::cout << rowPtr[i] << " ";
    std::cout << "\ncolIdx: ";
    for (int i = 0; i < nnz; ++i) std::cout << colIndicesOut[i] << " ";
    std::cout << "\nvalues: ";
    for (int i = 0; i < nnz; ++i) std::cout << valuesOut[i] << " ";
    std::cout << std::endl;

    cudaFree(d_rowIndices);
    cudaFree(d_colIndices);
    cudaFree(d_values);
    cudaFree(d_rowPtr);
    delete[] rowPtr;
    delete[] colIndicesOut;
    delete[] valuesOut;
}

int main() {
    const char *filename = "output.mtx";
    mtxToCsr(filename);
    return 0;
}


rowPtr: 0 1 2 3 5 
colIdx: 2 0 1 0 3 
values: 3 22 17 8 10 



In [ ]:
%%cuda
#include <cuda_runtime.h>
#include <iostream>
#include <algorithm>

// Kernel to count the number of non-zero elements per column
__global__ void countNonZerosPerColumn(const int *colIdx, int nnz, int *colCount) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < nnz) {
        atomicAdd(&colCount[colIdx[idx]], 1);
    }
}

// Kernel to compute column pointers (cumulative sum)
__global__ void computeColPtr(int *colCount, int *colPtr, int cols) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx == 0) {
        colPtr[0] = 0;
        for (int i = 1; i <= cols; ++i) {
            colPtr[i] = colPtr[i - 1] + colCount[i - 1];
        }
    }
}

// Kernel to rearrange data to CSC format
__global__ void rearrangeToCSC(const int *rowPtr, const int *colIdx, const float *values, int nnz,
                               int *rowIndicesCSC, float *valuesCSC, int *colPtr) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < nnz) {
        int col = colIdx[idx];
        int dstIdx = atomicAdd(&colPtr[col], 1);
        rowIndicesCSC[dstIdx] = rowPtr[idx];
        valuesCSC[dstIdx] = values[idx];
    }
}

void csrToCsc(const int *rowPtr, const int *colIdx, const float *values, int rows, int cols, int nnz) {
    int *d_rowPtr, *d_colIdx, *d_colCount, *d_colPtr, *d_rowIndicesCSC;
    float *d_values, *d_valuesCSC;

    // Allocate memory on the GPU
    cudaMalloc(&d_rowPtr, (rows + 1) * sizeof(int));
    cudaMalloc(&d_colIdx, nnz * sizeof(int));
    cudaMalloc(&d_values, nnz * sizeof(float));
    cudaMalloc(&d_colCount, cols * sizeof(int));
    cudaMalloc(&d_colPtr, (cols + 1) * sizeof(int));
    cudaMalloc(&d_rowIndicesCSC, nnz * sizeof(int));
    cudaMalloc(&d_valuesCSC, nnz * sizeof(float));

    // Copy CSR data to the GPU
    cudaMemcpy(d_rowPtr, rowPtr, (rows + 1) * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_colIdx, colIdx, nnz * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_values, values, nnz * sizeof(float), cudaMemcpyHostToDevice);

    cudaMemset(d_colCount, 0, cols * sizeof(int));

    // Count non-zeros per column
    int threadsPerBlock = 256;
    int blocks = (nnz + threadsPerBlock - 1) / threadsPerBlock;
    countNonZerosPerColumn<<<blocks, threadsPerBlock>>>(d_colIdx, nnz, d_colCount);

    // Compute column pointers (exclusive scan)
    computeColPtr<<<1, 1>>>(d_colCount, d_colPtr, cols);

    // Rearrange data to CSC
    rearrangeToCSC<<<blocks, threadsPerBlock>>>(d_rowPtr, d_colIdx, d_values, nnz, d_rowIndicesCSC, d_valuesCSC, d_colPtr);

    // Copy results back to the host for verification
    int *rowIndicesCSC = new int[nnz];
    float *valuesCSC = new float[nnz];
    int *colPtr = new int[cols + 1];
    cudaMemcpy(rowIndicesCSC, d_rowIndicesCSC, nnz * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(valuesCSC, d_valuesCSC, nnz * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(colPtr, d_colPtr, (cols + 1) * sizeof(int), cudaMemcpyDeviceToHost);

    // Output the CSC representation for verification
    std::cout << "colPtr: ";
    for (int i = 0; i <= cols; ++i) std::cout << colPtr[i] << " ";
    std::cout << "\nrowIdx: ";
    for (int i = 0; i < nnz; ++i) std::cout << rowIndicesCSC[i] << " ";
    std::cout << "\nvalues: ";
    for (int i = 0; i < nnz; ++i) std::cout << valuesCSC[i] << " ";
    std::cout << std::endl;

    // Free allocated memory
    cudaFree(d_rowPtr);
    cudaFree(d_colIdx);
    cudaFree(d_values);
    cudaFree(d_colCount);
    cudaFree(d_colPtr);
    cudaFree(d_rowIndicesCSC);
    cudaFree(d_valuesCSC);
    delete[] rowIndicesCSC;
    delete[] valuesCSC;
    delete[] colPtr;
}

int main() {
    int rowPtr[] = {0, 1, 2, 3, 5};
    int colIdx[] = {2, 0, 1, 0, 3};
    float values[] = {3, 22, 17, 8, 10};
    int nnz = 5;
    int rows = 4, cols = 4;

    csrToCsc(rowPtr, colIdx, values, rows, cols, nnz);
    return 0;
}


colPtr: 2 3 4 5 5 
rowIdx: 1 3 2 0 5 
values: 22 8 17 3 10 



In [ ]:
%%cuda
#include <cuda_runtime.h>
#include <iostream>

// Kernel to count the number of non-zero elements per row
__global__ void countNonZerosPerRow(const int *rowIdx, int nnz, int *rowCount) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < nnz) {
        atomicAdd(&rowCount[rowIdx[idx]], 1);
    }
}

// Kernel to compute row pointers (cumulative sum)
__global__ void computeRowPtr(int *rowCount, int *rowPtr, int rows) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx == 0) {
        rowPtr[0] = 0;
        for (int i = 1; i <= rows; ++i) {
            rowPtr[i] = rowPtr[i - 1] + rowCount[i - 1];
        }
    }
}

// Kernel to rearrange data to CSR format
__global__ void rearrangeToCSR(const int *colPtr, const int *rowIdx, const float *values, int nnz,
                               int *colIndicesCSR, float *valuesCSR, int *rowPtr) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < nnz) {
        int row = rowIdx[idx];
        int dstIdx = atomicAdd(&rowPtr[row], 1);
        colIndicesCSR[dstIdx] = colPtr[idx];
        valuesCSR[dstIdx] = values[idx];
    }
}

void cscToCsr(const int *colPtr, const int *rowIdx, const float *values, int rows, int cols, int nnz) {
    int *d_colPtr, *d_rowIdx, *d_rowCount, *d_rowPtr, *d_colIndicesCSR;
    float *d_values, *d_valuesCSR;

    // Allocate memory on the GPU
    cudaMalloc(&d_colPtr, (cols + 1) * sizeof(int));
    cudaMalloc(&d_rowIdx, nnz * sizeof(int));
    cudaMalloc(&d_values, nnz * sizeof(float));
    cudaMalloc(&d_rowCount, rows * sizeof(int));
    cudaMalloc(&d_rowPtr, (rows + 1) * sizeof(int));
    cudaMalloc(&d_colIndicesCSR, nnz * sizeof(int));
    cudaMalloc(&d_valuesCSR, nnz * sizeof(float));

    // Copy CSC data to the GPU
    cudaMemcpy(d_colPtr, colPtr, (cols + 1) * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_rowIdx, rowIdx, nnz * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_values, values, nnz * sizeof(float), cudaMemcpyHostToDevice);

    cudaMemset(d_rowCount, 0, rows * sizeof(int));

    // Count non-zeros per row
    int threadsPerBlock = 256;
    int blocks = (nnz + threadsPerBlock - 1) / threadsPerBlock;
    countNonZerosPerRow<<<blocks, threadsPerBlock>>>(d_rowIdx, nnz, d_rowCount);

    // Compute row pointers (exclusive scan)
    computeRowPtr<<<1, 1>>>(d_rowCount, d_rowPtr, rows);

    // Rearrange data to CSR
    rearrangeToCSR<<<blocks, threadsPerBlock>>>(d_colPtr, d_rowIdx, d_values, nnz, d_colIndicesCSR, d_valuesCSR, d_rowPtr);

    // Copy results back to the host for verification
    int *rowPtr = new int[rows + 1];
    int *colIndicesCSR = new int[nnz];
    float *valuesCSR = new float[nnz];
    cudaMemcpy(rowPtr, d_rowPtr, (rows + 1) * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(colIndicesCSR, d_colIndicesCSR, nnz * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(valuesCSR, d_valuesCSR, nnz * sizeof(float), cudaMemcpyDeviceToHost);

    // Output the CSR representation for verification
    std::cout << "rowPtr: ";
    for (int i = 0; i <= rows; ++i) std::cout << rowPtr[i] << " ";
    std::cout << "\ncolIdx: ";
    for (int i = 0; i < nnz; ++i) std::cout << colIndicesCSR[i] << " ";
    std::cout << "\nvalues: ";
    for (int i = 0; i < nnz; ++i) std::cout << valuesCSR[i] << " ";
    std::cout << std::endl;

    // Free allocated memory
    cudaFree(d_colPtr);
    cudaFree(d_rowIdx);
    cudaFree(d_values);
    cudaFree(d_rowCount);
    cudaFree(d_rowPtr);
    cudaFree(d_colIndicesCSR);
    cudaFree(d_valuesCSR);
    delete[] rowPtr;
    delete[] colIndicesCSR;
    delete[] valuesCSR;
}

int main() {
    int colPtr[] = {0, 2, 3, 4, 5};
    int rowIdx[] = {1, 3, 2, 0, 3};
    float values[] = {22, 8, 17, 3, 10};
    int nnz = 5;
    int rows = 4, cols = 4;

    cscToCsr(colPtr, rowIdx, values, rows, cols, nnz);
    return 0;
}


rowPtr: 1 2 3 5 5 
colIdx: 4 0 3 2 5 
values: 3 22 17 8 10 

